## The datasets
### 1. Socioeconomic Indicators in Chicago

This dataset contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.

### 2. Chicago Public Schools

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. This dataset is provided by the city of Chicago's Data Portal.

### 3. Chicago Crime Data

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days. 

### Download and store the datasets in databse tables as CENSUS_DATA, CHICAGO_PUBLIC_SCHOOLS & CHICAGO_CRIME_DATA

### Connect to the database

In [1]:
%load_ext sql

In [2]:
%sql ibm_db_sa://hgq74853:k%5E8r49r476tb49w1@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB

DB2/LINUXX8664


#### Finding total number of crimes recorded in the CRIME_DATA table

In [8]:
%%sql 
SELECT COUNT(*) AS Total_Crimes 
FROM CHICAGO_CRIME_DATA

 * ibm_db_sa://hgq74853:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


total_crimes
533


##### Listing  community areas with per capita income less than 11000.

In [9]:
%%sql 
SELECT community_area_name, per_capita_income 
FROM CENSUS_DATA 
WHERE per_capita_income < 11000

 * ibm_db_sa://hgq74853:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,per_capita_income
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


##### Listing all case numbers for crimes involving minors

In [27]:
%%sql
SELECT case_number, description
FROM CHICAGO_CRIME_DATA
WHERE description like '%MINOR%'

 * ibm_db_sa://hgq74853:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


case_number,description
HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,ILLEGAL CONSUMPTION BY MINOR


##### Listing all kidnapping crimes involving a child

In [52]:
%%sql
SELECT *
FROM CHICAGO_CRIME_DATA
WHERE primary_type = 'KIDNAPPING' AND description like '%CHILD%'

 * ibm_db_sa://hgq74853:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


id,case_number,DATE,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area_number,fbicode,x_coordinate,y_coordinate,YEAR,latitude,longitude,location
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,FALSE,FALSE,1533,15,29,25,20,1143050,1897546,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


##### Listing crimes were recorded at schools

In [22]:
%%sql
SELECT DISTINCT primary_type
FROM CHICAGO_CRIME_DATA
WHERE location_description like 'SCHOOL%'

 * ibm_db_sa://hgq74853:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


primary_type
ASSAULT
BATTERY
CRIMINAL DAMAGE
CRIMINAL TRESPASS
NARCOTICS
PUBLIC PEACE VIOLATION


##### Average safety score for all types of schools

In [33]:
%%sql
SELECT school_type, avg(safety_score) AS avg_safety_score
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY school_type

 * ibm_db_sa://hgq74853:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


school_type,avg_safety_score
ES,49.520383
HS,49.623529
MS,48.000000


##### Listing 5 community areas with highest % of households below poverty line

In [35]:
%%sql
SELECT community_area_name, percent_households_below_poverty
FROM CENSUS_DATA
ORDER BY percent_households_below_poverty DESC
LIMIT 5

 * ibm_db_sa://hgq74853:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,percent_households_below_poverty
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


##### Community area that is most crime prone

In [50]:
%%sql
SELECT community_area_number, count(community_area_number) AS total_crimes
FROM CHICAGO_CRIME_DATA
GROUP BY community_area_number
ORDER BY total_crimes DESC
LIMIT 1

 * ibm_db_sa://hgq74853:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_number,total_crimes
25,43


##### Using sub-query to find the name of the community area with highest hardship index

In [51]:
%%sql
SELECT community_area_name, hardship_index
FROM CENSUS_DATA
WHERE hardship_index = (SELECT MAX(hardship_index) FROM CENSUS_DATA)

 * ibm_db_sa://hgq74853:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,hardship_index
Riverdale,98


### Problem 10

##### Use a sub-query to determine the Community Area Name with most number of crimes?


In [48]:
%%sql
SELECT community_area_name
FROM CENSUS_DATA
WHERE community_area_number = (SELECT community_area_number FROM
                                (SELECT community_area_number, count(community_area_number)
                                FROM CHICAGO_CRIME_DATA
                                GROUP BY community_area_number
                                ORDER BY 2 DESC
                                LIMIT 1) AS temptable)

 * ibm_db_sa://hgq74853:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name
Austin
